In [1]:
#!pip install yfinance

In [2]:
#!pip install plotly

In [40]:
import warnings
warnings.filterwarnings('ignore')

import math
import polars as pl
import sklearn as sklearn
import scipy as scipy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
import tensorflow.keras as keras
import tensorflow as tf
import cv2 as opencv
import numpy as np
import yfinance

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Add, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import plotly
from plotly.graph_objs import *
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error, explained_variance_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [50]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [85]:
def get_yf_hist(symbol_list,startDate,endDate,interval):
  import matplotlib.pyplot as plt
  import datetime
  import yfinance as yf

  # Get historical pricing data
  data = yf.download(symbol_list, startDate, endDate, interval)

  return data


def get_intraday(symbol_list,period,interval):
  intraday_list = []
  intraday_df = pd.DataFrame()
  for sym in symbol_list:
    intraday = yf.download(tickers=sym,
                           period=period,
                           interval=interval)
    intraday['symbol'] = sym
    intraday_df = pd.concat([intraday_df,intraday])

  return intraday_df

def rolling_zscore(data,return_period,window_length):
  log_returns = (np.log(data / data.shift(return_period)))
  zscore = (log_returns - log_returns.rolling(window_length).mean() / log_returns.rolling(window_length).std())
  #results_dict = dict({'log_returns':log_returns})
  results_df = pd.DataFrame(zscore)

  return results_df


  # convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def rmse(y_true,y_pred):
    
    # Ensure both lists have the same length
    if len(y_true) != len(y_pred):
        raise ValueError("The length of actual and predicted values must be the same")
    
    # Calculate the squared differences
    squared_diff = [(yt-yp) ** 2 for yt, yp in zip(y_true,y_pred)]
    
    # Calculate mean of the squared differences
    mean_squared_diff = sum(squared_diff) / len(y_true)
    
    # Return the square root of the mean squared difference
    rmse = math.sqrt(mean_squared_diff)
    
    return rmse

In [86]:
#symbol_list = ['GS','JPM','C','WFC','STT']
#symbol_list = ['^GSPC','AMD','NVDA','TXN']
#symbol_list = ['^GSPC','SPY','MSFT','AAPL','NVDA','AMZN','LLY','JPM','AMD','XOM','UNH','PG','JNJ','HD','MRK','COST','CVX','NFLX','CRM','WMT','AMD','BAC','PEP','KO','LIN','ADBE','WFC','TMO','DIS','ACN','MCD','CSCO','ORCL','ABT','CAT','QCOM','INTU','AMGN','BKNG','LOW','ISRG','CRM']

#symbol_list = ['MA','V','^GSPC']
#symbol_list = ['^GSPC','UNH','GS','^DJI','MSFT','HD','SHW','CAT','CRM','V','AMGN','MCD','AXP','TRV','JPM','HON','AAPL','AMZN','IBM','PG','CVX','JNJ','NVDA','BA','MMM','DIS','MRK','WMT','NKE','KO','CSCO','VZ']

symbol_list = ['^GSPC','^DJI','ZN=F','ES=F','^RUT']

startDate = '1900-01-01'
endDate = '2024-11-15'

close_df = get_yf_hist(symbol_list,startDate,endDate,interval='1d')

[*********************100%***********************]  5 of 5 completed


In [87]:
close_df = close_df['Adj Close']

target_col = '^RUT'

def structure_data(symbol_list,close_df,target_col):
    if len(symbol_list) > 1:
        special_col = target_col
        cols = [special_col] + [col for col in close_df.columns if col != special_col]
        close_df = close_df[cols]
        return close_df
    else:
        return close_df

close_df = structure_data(symbol_list,close_df,target_col)

period_list = [30,60,90,120,252,365]

def add_predictors(col_name,df,period_list):
    for period in period_list:
        df[f"{col_name}_{period}_avg"] = df[col_name].rolling(period).mean()
        df[f"{col_name}_{period}_ratio"] = (df[col_name] / df[f"{col_name}_{period}_avg"])
    return df
    
close_df = add_predictors(col_name=target_col,period_list=period_list,df=close_df)

#close_df.head()
print(close_df.info())
close_df.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24366 entries, 1927-12-30 00:00:00+00:00 to 2024-11-14 00:00:00+00:00
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ^RUT            9370 non-null   float64
 1   ES=F            6103 non-null   float64
 2   ZN=F            6065 non-null   float64
 3   ^DJI            8280 non-null   float64
 4   ^GSPC           24336 non-null  float64
 5   ^RUT_30_avg     8628 non-null   float64
 6   ^RUT_30_ratio   8628 non-null   float64
 7   ^RUT_60_avg     8156 non-null   float64
 8   ^RUT_60_ratio   8156 non-null   float64
 9   ^RUT_90_avg     7950 non-null   float64
 10  ^RUT_90_ratio   7950 non-null   float64
 11  ^RUT_120_avg    7813 non-null   float64
 12  ^RUT_120_ratio  7813 non-null   float64
 13  ^RUT_252_avg    7290 non-null   float64
 14  ^RUT_252_ratio  7290 non-null   float64
 15  ^RUT_365_avg    7063 non-null   float64
 16  ^RUT_365_ratio  7063 non-null

Ticker,^RUT,ES=F,ZN=F,^DJI,^GSPC,^RUT_30_avg,^RUT_30_ratio,^RUT_60_avg,^RUT_60_ratio,^RUT_90_avg,^RUT_90_ratio,^RUT_120_avg,^RUT_120_ratio,^RUT_252_avg,^RUT_252_ratio,^RUT_365_avg,^RUT_365_ratio
Date,,,,,,,,,,,,,,,,,
2024-11-08 00:00:00+00:00,2399.639893,6025.25,110.281250,43988.988281,5995.540039,2241.920662,1.070350,2209.424154,1.086093,2187.890103,1.096783,2151.451410,1.115359,NaN,NaN,NaN,NaN
2024-11-11 00:00:00+00:00,2434.979980,6031.75,109.906250,44293.128906,6001.350098,2248.754329,1.082813,2214.308488,1.099657,2192.426214,1.110633,2154.256576,1.130311,NaN,NaN,NaN,NaN
2024-11-12 00:00:00+00:00,2391.850098,6013.00,109.421875,43910.980469,5983.990234,2255.248332,1.060571,2218.047656,1.078358,2196.350437,1.089011,2156.841077,1.108960,NaN,NaN,NaN,NaN
2024-11-13 00:00:00+00:00,2369.370117,6016.00,109.421875,43958.191406,5985.379883,2261.060335,1.047902,2221.833993,1.066403,2200.127105,1.076924,2159.515745,1.097177,NaN,NaN,NaN,NaN
2024-11-14 00:00:00+00:00,2336.939941,5978.25,109.546875,43750.859375,5949.169922,2266.286670,1.031176,2224.606991,1.050496,2203.295882,1.060656,2161.742995,1.081044,NaN,NaN,NaN,NaN


In [88]:
close_corrs_df = close_df.corr()
close_corrs_diffs_df = close_df.diff()
close_corrs_diffs_df = close_corrs_diffs_df.corr()

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(z=close_corrs_df.values,
                                 x=close_corrs_df.columns.tolist(),
                                 y=close_corrs_df.index.tolist(),
                                 showscale=True,
                                 colorscale='Viridis')
fig.show()

In [89]:
close_corrs_diffs_df.sort_values(by=target_col,ascending=False)

Ticker,^RUT,ES=F,ZN=F,^DJI,^GSPC,^RUT_30_avg,^RUT_30_ratio,^RUT_60_avg,^RUT_60_ratio,^RUT_90_avg,^RUT_90_ratio,^RUT_120_avg,^RUT_120_ratio,^RUT_252_avg,^RUT_252_ratio,^RUT_365_avg,^RUT_365_ratio
Ticker,,,,,,,,,,,,,,,,,
^RUT,1.000000,0.830129,-0.172168,0.816612,0.841596,0.171744,0.836049,0.126983,0.848193,0.097749,0.854432,0.099404,0.856679,0.056254,0.863627,0.048865,0.863148
^RUT_252_ratio,0.863627,0.758378,-0.279958,0.727053,0.743983,0.140467,0.974186,0.085563,0.985893,0.058209,0.991432,0.049223,0.994537,0.000487,1.000000,0.003831,0.998222
^RUT_365_ratio,0.863148,0.757474,-0.272641,0.721914,0.740354,0.144857,0.969748,0.093966,0.981774,0.069728,0.987641,0.063139,0.991110,0.018799,0.998222,0.004215,1.000000
^RUT_120_ratio,0.856679,0.751154,-0.277515,0.730513,0.740654,0.122150,0.982224,0.062059,0.993269,0.028271,0.997839,0.011906,1.000000,0.003210,0.994537,0.006985,0.991110
^RUT_90_ratio,0.854432,0.749202,-0.272165,0.731045,0.740117,0.111931,0.984774,0.044527,0.995875,0.003471,1.000000,0.013932,0.997839,0.003351,0.991432,0.007319,0.987641
^RUT_60_ratio,0.848193,0.741956,-0.265616,0.729355,0.735419,0.086855,0.989331,0.002384,1.000000,0.004930,0.995875,0.013562,0.993269,0.003100,0.985893,0.004915,0.981774
^GSPC,0.841596,0.977909,-0.177843,0.937104,1.000000,0.143447,0.728402,0.096653,0.735419,0.080384,0.740117,0.085796,0.740654,0.065864,0.743983,0.047092,0.740354
^RUT_30_ratio,0.836049,0.733251,-0.252766,0.725285,0.728402,0.004971,1.000000,0.003567,0.989331,0.005003,0.984774,0.010465,0.982224,0.002983,0.974186,0.002142,0.969748
ES=F,0.830129,1.000000,-0.174418,0.916136,0.977909,0.145201,0.733251,0.098648,0.741956,0.081720,0.749202,0.087860,0.751154,0.067277,0.758378,0.047099,0.757474


In [90]:
# This code helps you visualize how many null values are in each column
print(f"NULL value counts: {pd.isnull(close_df).sum().sort_values(ascending=False)}")
print("")
print(f"NULL Value percentages: {(pd.isnull(close_df).sum()).sort_values(ascending=False) / len(close_df)*100}")

df2 = close_df
df2 = df2.dropna(axis=0)
df2 = df2.sort_index(ascending=True)

print(f'df2 shape: {df2.shape}')
print(f'df2 head: {df2.head()}')

df2_values = df2.values
df2_values = df2_values.astype('float32')

#n_out variable is the number of periods to forecast outward in time
#n_in variable is the number of periods to lag the time series of each variable

df2_fmt = series_to_supervised(df2_values,1,1)

f"df2_values shape: {df2_values.shape}"

df2_fmt2 = df2_fmt.iloc[:,:-len(df2.columns)+1]

print(f"df2_fmt shape: {df2_fmt2.shape}")
print(f"df2_fmt2 head: {df2_fmt2.head()}")

n_in = 1

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(df2_values)
df2_reframed = series_to_supervised(scaled,n_in,1)
df2_reframed3 = series_to_supervised(scaled,n_in,1)

df2_reframed = df2_reframed.iloc[:,:-len(df2.columns)]
df2_reframed['var1(t)'] = df2_reframed3['var1(t)']

print(f"df2_reframed head {df2_reframed.head()}")
# Now we have the primary stock we want to prdict (var1), its prices shifted back by 1 unit (because that was our chosen input value for our supervised learning function), as well as the other stocks that are also shifted all in the same dataframe

values = df2_reframed.values

train_split = round(int(len(values)*0.60))
test_split = (len(values) - train_split)

train = values[:train_split]
test = values[train_split:]

X_train,y_train = train[:,:-1],train[:,-1]
X_test,y_test = test[:,:-1],test[:,-1]

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}, X_test shape: {X_test.shape},y_test shape: {y_test.shape}")

# reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

X_train.shape
X_test.shape

df = close_df

NULL value counts: Ticker
ZN=F              18301
ES=F              18263
^RUT_365_ratio    17303
^RUT_365_avg      17303
^RUT_252_ratio    17076
^RUT_252_avg      17076
^RUT_120_ratio    16553
^RUT_120_avg      16553
^RUT_90_ratio     16416
^RUT_90_avg       16416
^RUT_60_ratio     16210
^RUT_60_avg       16210
^DJI              16086
^RUT_30_ratio     15738
^RUT_30_avg       15738
^RUT              14996
^GSPC                30
dtype: int64

NULL Value percentages: Ticker
ZN=F              75.108758
ES=F              74.952803
^RUT_365_ratio    71.012887
^RUT_365_avg      71.012887
^RUT_252_ratio    70.081261
^RUT_252_avg      70.081261
^RUT_120_ratio    67.934827
^RUT_120_avg      67.934827
^RUT_90_ratio     67.372568
^RUT_90_avg       67.372568
^RUT_60_ratio     66.527128
^RUT_60_avg       66.527128
^DJI              66.018222
^RUT_30_ratio     64.590002
^RUT_30_avg       64.590002
^RUT              61.544776
^GSPC              0.123122
dtype: float64
df2 shape: (4121, 17)
df2 head

In [91]:
# Parameters
patience = 50
#sequence_length = 30  # Number of days to look back
features = df.shape[1]  # Number of features
epochs = 200
batch_size = 16
# The below filter size seems to have the best results thus far
#filter1 = 128

# This is our new experimental filter value
#filter1 = 64
filter1 = 10

early_stopping = EarlyStopping(patience=patience,monitor='val_loss',restore_best_weights=True)
checkpoint_filepath = "/kaggle/working Checkpoints/best_model{epoch:02d}_{val_loss:.2f}.keras"
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=(patience/2),verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0.0)

# df = close_df

# TCN model architecture
def build_tcn(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(filter1, kernel_size=5, dilation_rate=1, padding="causal")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Stacking TCN blocks with residual connections
    for dilation_rate in [2, 4, 6, 8, 16, 64]:
        previous_x = x
        x = Conv1D(filter1, kernel_size=5, dilation_rate=dilation_rate, padding="causal")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Add()([x, previous_x])  # Residual connection
        x = Dropout(rate=0.3)(x)
    
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    return model

# Compile model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_tcn(input_shape)
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[early_stopping,reduce_lr])

# Evaluate model
mse = model.evaluate(X_test, y_test)
print("Test MSE:", mse)

predictions = model.predict(X_test)

Epoch 1/200
155/155 [==============================] - 12s 20ms/step - loss: 4.6253 - val_loss: 0.4623 - lr: 0.0100
Epoch 2/200
155/155 [==============================] - 3s 17ms/step - loss: 0.1886 - val_loss: 0.2647 - lr: 0.0100
Epoch 3/200
155/155 [==============================] - 3s 19ms/step - loss: 0.0539 - val_loss: 0.1941 - lr: 0.0100
Epoch 4/200
155/155 [==============================] - 3s 19ms/step - loss: 0.0251 - val_loss: 0.1520 - lr: 0.0100
Epoch 5/200
155/155 [==============================] - 3s 19ms/step - loss: 0.0189 - val_loss: 0.1096 - lr: 0.0100
Epoch 6/200
155/155 [==============================] - 3s 19ms/step - loss: 0.0179 - val_loss: 0.0991 - lr: 0.0100
Epoch 7/200
155/155 [==============================] - 3s 19ms/step - loss: 0.0175 - val_loss: 0.1187 - lr: 0.0100
Epoch 8/200
155/155 [==============================] - 3s 18ms/step - loss: 0.0176 - val_loss: 0.1219 - lr: 0.0100
Epoch 9/200
155/155 [==============================] - 3s 19ms/step - loss: 0.0

Epoch 70/200
155/155 [==============================] - 3s 19ms/step - loss: 0.0060 - val_loss: 0.1089 - lr: 1.0000e-03
Epoch 71/200
154/155 [============================>.] - ETA: 0s - loss: 0.0062
Epoch 71: ReduceLROnPlateau reducing learning rate to 9.999999310821295e-05.
52/52 [==============================] - 1s 22ms/step - loss: 0.0054
Test MSE: 0.005434055346995592
52/52 [==============================] - 0s 5ms/step


In [79]:
trace_0 = go.Scatter(y=history.history['loss'],
                     name='train',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=history.history['val_loss'],
                     name='test',
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]

layout = go.Layout(title="train/test logarithmic loss comparison",
                  template='plotly_dark',
                   height=600,
                   width=1000)

fig = go.Figure(data,layout=layout)
fig.show()

In [94]:
# Deploy the model, return predictions, and reformat the X_test data
predicted = model.predict(X_test)
X_test_rs = X_test.reshape(X_test.shape[0],X_test.shape[2])

# Rescale the test data
new_df = pd.DataFrame(X_test_rs[:,1:])
new_df2 = pd.concat([pd.DataFrame(y_test),pd.DataFrame(new_df)],axis=1)
new_df2.columns = df2.columns
new_df3 = scaler.inverse_transform(new_df2)
new_df3 = pd.DataFrame(new_df3,columns=df2.columns)

# Rescale the predictions data
new_pred_df = pd.DataFrame(predicted)
new_pred = pd.concat([new_pred_df,pd.DataFrame(new_df)],axis=1)
new_pred = scaler.inverse_transform(new_pred)
new_pred = pd.DataFrame(new_pred,columns=df2.columns)

# Create a DataFrame with the predicted and actual values and take the difference between the two
results_df = pd.concat([new_pred.iloc[:,0],new_df3.iloc[:,0]],axis=1)
results_df.columns = ['theta_hat','theta']
results_df['Difference'] = results_df['theta_hat'] - results_df['theta']

print(f"Root Mean Squared Error: {rmse(results_df['theta_hat'],results_df['theta'])}")

results_df

52/52 [==============================] - 0s 9ms/step
Root Mean Squared Error: 154.7653617559253


,theta_hat,theta,Difference
0,1162.697510,1391.859985,-229.162476
1,1161.151855,1399.589966,-238.438110
2,1161.464233,1390.199951,-228.735718
3,1161.482422,1382.770020,-221.287598
4,1164.177002,1393.920044,-229.743042
...,...,...,...
1643,1675.780762,1773.760010,-97.979248
1644,1672.840698,1797.770020,-124.929321
1645,1678.069580,1807.079956,-129.010376
1646,1685.656616,1783.260010,-97.603394


In [95]:
trace_0 = go.Scatter(y=results_df['theta_hat'],
                     name='predicted',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=results_df['theta'],
                     name=df2.columns[0],
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]


layout = go.Layout(title=f"TCN model predictions vs. actual price of {df2.columns[0]}",
                  template='plotly_dark',
                   height=900,
                   width=1500)

fig = go.Figure(data,layout=layout)
fig.show()

In [96]:
# def reset_model(model):
#   del model
#   model = Sequential()
#   return model

# model = reset_model(model)
print(explained_variance_score(y_true=results_df['theta'],y_pred=results_df['theta_hat']))

0.8503724932670593


The below model has multiple TCN layers instead of only one like the above model

In [100]:
# Parameters
patience = 50
sequence_length = 90  # Number of days to look back
features = df.shape[1]  # Number of features
epochs = 3
batch_size = 16
# The below filter size seems to have the best results thus far
#filter1 = 128

# This is our new experimental filter value
filter1 = 150


early_stopping = EarlyStopping(patience=patience,monitor='val_loss',restore_best_weights=True)
checkpoint_filepath = "/kaggle/working Checkpoints/best_model{epoch:02d}_{val_loss:.2f}.keras"
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=(patience/2),verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0.0)



def build_tcn(input_shape, num_layers_per_block=2):
    inputs = Input(shape=input_shape)
    
    # Initial Conv1D layer
    x = Conv1D(filter1, kernel_size=2, dilation_rate=1, padding="causal")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Stacked TCN blocks with multiple Conv1D layers
    for dilation_rate in [2, 4, 8, 16]:
        previous_x = x
        for _ in range(num_layers_per_block):  # Stack `num_layers_per_block` Conv1D layers
            x = Conv1D(filter1, kernel_size=2, dilation_rate=dilation_rate, padding="causal")(x)
            x = BatchNormalization()(x)
            x = ReLU()(x)
        
        # Add a residual connection to the block's input
        x = Add()([x, previous_x])
    
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    return model

# Compile and use the model as before
model = build_tcn(input_shape)
model.compile(optimizer=Adam(), loss='mean_squared_error')
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[early_stopping,reduce_lr])


Epoch 1/3
155/155 [==============================] - 8s 25ms/step - loss: 0.5331 - val_loss: 0.3142 - lr: 0.0010
Epoch 2/3
155/155 [==============================] - 5s 31ms/step - loss: 0.1165 - val_loss: 0.1119 - lr: 0.0010
Epoch 3/3
155/155 [==============================] - 3s 21ms/step - loss: 0.0692 - val_loss: 0.1585 - lr: 0.0010


In [98]:
trace_0 = go.Scatter(y=history.history['loss'],
                     name='train',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=history.history['val_loss'],
                     name='test',
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]

layout = go.Layout(title="train/test logarithmic loss comparison",
                  template='plotly_dark',
                   height=600,
                   width=1000)

fig = go.Figure(data,layout=layout)
fig.show()

In [29]:
# Deploy the model, return predictions, and reformat the X_test data
predicted = model.predict(X_test)
X_test_rs = X_test.reshape(X_test.shape[0],X_test.shape[2])

# Rescale the test data
new_df = pd.DataFrame(X_test_rs[:,1:])
new_df2 = pd.concat([pd.DataFrame(y_test),pd.DataFrame(new_df)],axis=1)
new_df2.columns = df2.columns
new_df3 = scaler.inverse_transform(new_df2)
new_df3 = pd.DataFrame(new_df3,columns=df2.columns)

# Rescale the predictions data
new_pred_df = pd.DataFrame(predicted)
new_pred = pd.concat([new_pred_df,pd.DataFrame(new_df)],axis=1)
new_pred = scaler.inverse_transform(new_pred)
new_pred = pd.DataFrame(new_pred,columns=df2.columns)

# Create a DataFrame with the predicted and actual values and take the difference between the two
results_df = pd.concat([new_pred.iloc[:,0],new_df3.iloc[:,0]],axis=1)
results_df.columns = ['theta_hat','theta']
results_df['Difference'] = results_df['theta_hat'] - results_df['theta']

print(f"Root Mean Squared Error: {rmse(results_df['theta_hat'],results_df['theta'])}")

results_df

66/66 [==============================] - 0s 5ms/step
Root Mean Squared Error: 55.81160885732388


,theta_hat,theta,Difference
0,86.689781,82.853264,3.836517
1,85.242088,83.788368,1.453720
2,86.539658,84.504028,2.035629
3,88.848427,84.685341,4.163086
4,89.264931,85.878082,3.386848
...,...,...,...
2084,632.880554,514.070007,118.810547
2085,618.866272,513.760010,105.106262
2086,609.295288,516.340027,92.955261
2087,615.517883,515.010010,100.507874


In [30]:
trace_0 = go.Scatter(y=results_df['theta_hat'],
                     name='predicted',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=results_df['theta'],
                     name=df2.columns[0],
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]


layout = go.Layout(title=f"LSTM model predictions vs. actual price of {df2.columns[0]}",
                  template='plotly_dark',
                   height=900,
                   width=1500)

fig = go.Figure(data,layout=layout)
fig.show()